In [1]:
#nueral network
import numpy as np
import pandas as pd
import os
import random
import time

# ML imports
from sklearn.model_selection import KFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, IncrementalPCA

# Visualizations
import matplotlib.pyplot as plt  # static plotting
import seaborn as sns  # pretty plotting, including heat map
%matplotlib inline
from sklearn.preprocessing import RobustScaler

In [2]:
# df_train4=pd.read_csv(r"data\NewTable08_Map_Crime_MY1020110801imblearn-year1-5.csv")
df_train4=pd.read_csv(r"data\NewTable08_Map_Crime_MY1020110801imblearn-6.csv")
df_train4_drop=df_train4.dropna()
df_train4.columns

Index(['num_row', 'Month', 'Month_La', 'Year', 'Area_unitID', 'Area_Unit_TA',
       'Day Of Week ID', 'Day_Of_Week', 'HRGroup', 'Time', 'TA_ID',
       'Locn_Type_DivisionID', 'Locn_Type_Division', 'Types of Crime ID',
       'Types_of_Crime', 'Crime_cases'],
      dtype='object')

In [3]:
df_train4_loaded=df_train4.drop(['num_row','Month','Year','Area_unitID','Day Of Week ID','HRGroup','TA_ID','Locn_Type_DivisionID','Types of Crime ID'],axis=1)
df_train4_loaded_drop=df_train4_loaded.dropna()

In [4]:
def generate_dict(data):
    my_dict= {v:k for k, v in enumerate(set(data))}
    return my_dict

In [5]:
d_Month_La=generate_dict(df_train4_loaded["Month_La"].unique())
d_Area_Unit=generate_dict(df_train4_loaded["Area_Unit_TA"].unique())
d_Day_Of_Week=generate_dict(df_train4_loaded["Day_Of_Week"].unique())
d_Time=generate_dict(df_train4_loaded["Time"].unique())
d_Locn_Type_Division=generate_dict(df_train4_loaded["Locn_Type_Division"].unique())
d_Types_of_Crime=generate_dict(df_train4_loaded_drop["Types_of_Crime"].unique())

In [6]:
df2=df_train4_loaded_drop
df2

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
0,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
1,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
2,November,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
3,January,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
4,February,"Mt Eden South, Auckland",UNKNOWN,00:00-23:59,Community Location,Theft and Related Offences,6
...,...,...,...,...,...,...,...
5453226,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453227,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9
5453228,March,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Not Applicable,"Unlawful Entry With Intent/Burglary, Break and...",9
5453229,June,"Pinehill, Auckland",UNKNOWN,00:00-23:59,Other Location,"Unlawful Entry With Intent/Burglary, Break and...",9


In [7]:
df2["Area_Unit_TA"]=df2["Area_Unit_TA"].map(d_Area_Unit)
df2["Month_La"]=df2["Month_La"].map(d_Month_La)
df2["Day_Of_Week"]=df2["Day_Of_Week"].map(d_Day_Of_Week)
df2["Time"]=df2["Time"].map(d_Time)
df2["Locn_Type_Division"]=df2["Locn_Type_Division"].map(d_Locn_Type_Division)
df2["Types_of_Crime"]=df2["Types_of_Crime"].map(d_Types_of_Crime)

C:\Users\admin\AppData\Local\Temp\ipykernel_17888\3222048269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Area_Unit_TA"]=df2["Area_Unit_TA"].map(d_Area_Unit)
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\3222048269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Month_La"]=df2["Month_La"].map(d_Month_La)
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\3222048269.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [8]:
df2["Month_La"] = df2['Month_La'].astype('int32')
df2["Area_Unit_TA"] = df2['Area_Unit_TA'].astype('int32')
df2["Day_Of_Week"] = df2['Day_Of_Week'].astype('int32')
df2["Time"] = df2['Time'].astype('int32')
df2["Locn_Type_Division"] = df2['Locn_Type_Division'].astype('int32')
df2["Types_of_Crime"] = df2['Types_of_Crime'].astype('int32')
df2["Crime_cases"] = df2['Crime_cases'].astype('int32')
df2.dtypes

C:\Users\admin\AppData\Local\Temp\ipykernel_17888\2869146917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Month_La"] = df2['Month_La'].astype('int32')
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\2869146917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Area_Unit_TA"] = df2['Area_Unit_TA'].astype('int32')
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\2869146917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

Month_La              int32
Area_Unit_TA          int32
Day_Of_Week           int32
Time                  int32
Locn_Type_Division    int32
Types_of_Crime        int32
Crime_cases           int32
dtype: object

In [9]:
df2.dtypes
df2_drop=df2.dropna()

In [10]:
df2_drop["Month_La"] = df2_drop['Month_La'].astype('int32')
df2_drop["Area_Unit_TA"] = df2_drop['Area_Unit_TA'].astype('int32')
df2_drop["Day_Of_Week"] = df2_drop['Day_Of_Week'].astype('int32')
df2_drop["Time"] = df2_drop['Time'].astype('int32')
df2_drop["Locn_Type_Division"] = df2_drop['Locn_Type_Division'].astype('int32')
df2_drop["Types_of_Crime"] = df2_drop['Types_of_Crime'].astype('int32')
df2_drop["Crime_cases"] = df2_drop['Crime_cases'].astype('int32')
df2_drop.dtypes

Month_La              int32
Area_Unit_TA          int32
Day_Of_Week           int32
Time                  int32
Locn_Type_Division    int32
Types_of_Crime        int32
Crime_cases           int32
dtype: object

In [11]:
df2_drop.head()

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
0,5,773,6,4,1,0,6
1,0,773,6,4,1,0,6
2,5,773,6,4,1,0,6
3,11,773,6,4,1,0,6
4,0,773,6,4,1,0,6


In [12]:
d_Month_La

{'February': 0,
 'August': 1,
 'May': 2,
 'October': 3,
 'April': 4,
 'November': 5,
 'September': 6,
 'December': 7,
 'July': 8,
 'March': 9,
 'June': 10,
 'January': 11}

In [13]:
df2_15 = df2_drop[df2_drop['Month_La'] != 6]
df2_test = df2_drop[df2_drop['Month_La'] == 6]

In [14]:
df2_test

,Month_La,Area_Unit_TA,Day_Of_Week,Time,Locn_Type_Division,Types_of_Crime,Crime_cases
109,6,773,6,4,1,0,4
116,6,773,6,4,0,2,5
119,6,773,6,4,1,0,4
130,6,773,6,4,1,0,4
134,6,773,6,4,0,2,5
...,...,...,...,...,...,...,...
5452734,6,1385,6,4,0,2,2
5452985,6,1385,1,4,0,2,1
5452995,6,1385,6,4,1,0,1
5453044,6,1385,5,4,3,0,1


In [15]:
target_name =  'Crime_cases'
robust_scaler = RobustScaler()
X=df2.drop(['Crime_cases'],axis=1)
y=df2[target_name]
# X_new=df2_test.drop(['Crime_cases'],axis=1)
# y_new=df2_test[target_name]
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [16]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=10, weights='distance',metric='euclidean',n_jobs=-1)
KNN=knn.fit(X_train,y_train)

In [17]:
knn.fit(X_train, y_train.values.ravel()),
print('Accuracy: ', knn.score(X_test, y_test)),

Accuracy:  0.790334534206495


(None,)

In [18]:
#User input
X=np.array([[2,1,1,2,1,1]])
X

array([[2, 1, 1, 2, 1, 1]])

In [19]:
pred = knn.predict(X_test)
print('F1 accuracy: ', f1_score(pred, y_test, average='macro'))
    # rfc_pred = rfc.predict(X_test)
print (classification_report(y_test,pred))

F1 accuracy:  0.7899996189154468
              precision    recall  f1-score   support

           1       0.95      0.73      0.83    109571
           2       0.77      0.76      0.76    108474
           3       0.75      0.73      0.74    108388
           4       0.74      0.73      0.74    108311
           5       0.76      0.76      0.76    109262
           6       0.76      0.77      0.77    108362
           7       0.78      0.83      0.80    108273
           8       0.82      0.83      0.82    108730
           9       0.81      0.92      0.86    108788
          10       0.81      0.84      0.82    109383

    accuracy                           0.79   1087542
   macro avg       0.79      0.79      0.79   1087542
weighted avg       0.79      0.79      0.79   1087542



In [20]:
import pickle

In [21]:
data= {"model":knn, 
       "d_Area_Unit":d_Area_Unit, 
       "d_Month_La":d_Month_La,"d_Day_Of_Week":d_Day_Of_Week,
       "d_Time":d_Time,
       "d_Locn_Type_Division":d_Locn_Type_Division,
       "d_Types_of_Crime":d_Types_of_Crime
      }

with open('save_steps.pkl','wb') as file:
    pickle.dump(data,file)

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
import sklearn.metrics as metrics

In [23]:
models = pd.DataFrame(index=['train_mse','test_mse','train_rmse','test_rmse'],
                     columns = ['knn'])

In [24]:
models.loc['train_mse','knn']= mean_squared_error(y_pred=knn.predict(X_train),y_true=y_train)
models.loc['test_mse','knn']= mean_squared_error(y_pred=knn.predict(X_test),y_true=y_test)
models.loc['train_rmse','knn']= np.sqrt(mean_squared_error(y_pred=knn.predict(X_train),y_true=y_train))
models.loc['test_rmse','knn']= np.sqrt(mean_squared_error(y_pred=knn.predict(X_test),y_true=y_test))

In [25]:
models

,knn
train_mse,1.354375
test_mse,1.51759
train_rmse,1.163776
test_rmse,1.231905


In [26]:
pred_prob = knn.predict_proba(X_test)
roc_auc=roc_auc_score(y_test, pred_prob, multi_class='ovr')

In [27]:
print('KNN roc AUC: ', roc_auc)

KNN roc AUC:  0.966386349738993
